In [1]:
from scipy.integrate import quad
from numpy import *
from matplotlib.pylab import *
import matplotlib as mpl
%matplotlib inline
#from pandas.core import *
import pandas as pd

import pickle
import scipy.interpolate as intpl

In [2]:
import sys
sys.path.append('/export/home1/users/bssn/serna/environments/neuron/bin',)

In [3]:
import salib 


In [4]:
import itertools

In [5]:
folder = "/mnt/data/spinemodel/conditions/"
condition = "1000_70_65"
#Jupyters: Proper Spine Model - Inhibition.ipynb
with open(folder+"dis_baselinev3_"+condition+"sh.pickle","rb") as f:
    vavgDiS,mesDiS,vtracsDiS,vtracsDDiS,vtracsSDiS,CtracsDiS = pickle.load(f)

with open(folder+"All_baseline_datasetv3_"+condition+"sh.pickle","rb") as f:
    vavgT,mesT,vtracsT,vtracsDT,vtracsST,CtracsT,me2T,_,dataT = pickle.load(f)

with open(folder+"dis_ga_glu02v3_"+condition+"sh.pickle","rb") as f:
    dis,mes1,mes01 = pickle.load(f)

with open(folder+"dis_glu_ga02v3_"+condition+"sh.pickle","rb") as f:
    dis,mes,mes0,times,times0 = pickle.load(f)

In [6]:
labels = ['A1','Dss','Rneck']#,'meanDhead']
labelsmod = labels
labels = ['A1','maxDhead','Dss','Lneck','Lhead','meanDneck_trunc70pc']
labelmorph = labels

In [7]:
dataT.keys(),vtracsST.shape

(dict_keys(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA', 'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck', 'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead', 'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0', 'Rmorph']),
 (10002, 384))

In [95]:
dataT['A1'].shape

(384,)

In [8]:
pddataT = pd.DataFrame(dataT, columns= list(dataT.keys()))

In [9]:
from bootstrap_unshr import btset_cd

nsp = 1000000
sps = btset_cd(cd='Sp',size=nsp)

In [10]:
sps.keys()

dict_keys(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA', 'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck', 'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead', 'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0'])

In [11]:
#dataA = np.column_stack((sps['A1'],sps['Dss'],sps['Rneck']))
# Me habia olvidado de la respuesta 
dataA = np.column_stack((mesT[:,1]-mesT[:,2],)+tuple(pddataT[key] 
                                               for key in labelsmod))

def rescale(x):
    xme, xst = x.mean(0, keepdims = True), x.std(0, keepdims = True)
    xt = (x-xme)/xst
    return xt, (xme,xst)

def undorescale(x,xme,xst):
    xt = x*xst+xme
    return xt

dataA, datpar = rescale(dataA)
print(undorescale(dataA,*datpar).mean(0),datpar[0])

[3.67245612e+00 1.44755168e-01 8.20235395e+01 2.11602387e+02] [[3.67245612e+00 1.44755168e-01 8.20235395e+01 2.11602387e+02]]


In [96]:
histogramdd?

In [160]:
hf = histogramdd(dataA, bins = (30,19,18,17), normed = True)
hxms = []
dx = 1.0
for i, hx in enumerate(hf[1]):
    hxm = (hx[1:]+hx[:-1])*.5
    hxms.append(hxm)
    dx = dx*(hx[1]-hx[0])
XX = meshgrid(*hxms, indexing='ij')
print(dx, hf[0].shape, *tuple(Xt.shape for Xt in XX))
var(dataA[:,2]), hf[0].sum()*dx, tuple((Xt**2*hf[0]).sum()/hf[0].sum() for Xt in XX)

0.009428709104247026 (30, 19, 18, 17) (30, 19, 18, 17) (30, 19, 18, 17) (30, 19, 18, 17) (30, 19, 18, 17)


(0.9999999999999997,
 1.0,
 (1.0027666064081076,
  1.0002768980372327,
  0.9953091881345394,
  0.9939385903751493))

In [161]:
h = hf[0]
hxs = {}
hxm = {}
dxs = {}
dx = 1.0
for i, hx in enumerate(hf[1]):
    hxs[i] = hx
    hxm[i] = (hx[1:]+hx[:-1])*.5
    dxs[i] = (hx[1]-hx[0]) 
    dx = dx*(hx[1]-hx[0])

In [162]:
ncol = 1
hfun = hxs[0]
nfun = h.shape[0]
h = hf[0]
# loop
hx = hxs[ncol]
nx = h.shape[ncol]

Eygx = np.zeros(nx)
for ix in range(nx):
    ht = np.take(h, ix, axis=ncol) # np.take(h,ix, axis = ncol)
    hxms = [hxm[i] for i in range(4) if i != ncol]

    XX = meshgrid(*hxms , indexing='ij')
    Eygx[ix] = sum(XX[0]*ht)/sum(ht)

ht = h.sum(axis = tuple([i for i in range(4) if i != ncol]))
hx = hxm[ncol]
xme = np.nansum(Eygx*ht/sum(ht))
xva = np.nansum((Eygx-xme)**2*ht/sum(ht))
print(xme, xva)

0.007073521423461335 0.4996727667199588


/export/home1/users/bssn/serna/environments/neuron/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [194]:
def sensitivityidcs(y, pd, labels, ylabel = 'Y', ns = None):
    alabels = [ylabel]+labels
    data = np.column_stack((y,)+tuple(pd[key] 
                                                   for key in labels))
    data, datpar = rescale(data)

    ncols = len(alabels)
    if ns is None:
        ns = [21]*ncols
        
    # We create n-dimensional histogram
    hf = histogramdd(data, bins = ns, normed = True)

    
    # Histogram
    h = hf[0]
    hxs = {}
    hxm = {}
    # x breaks
    # print(h.shape)
    me = 0.0
    va = 0.0
    dx = 1.0
    dxs = {}
    for i, hx in enumerate(hf[1]):
        hxs[i] = hx
        hxm[i] = (hx[1:]+hx[:-1])*.5
        dxs[i] = hx[1]-hx[0]
        dx = dx*(hx[1]-hx[0])
    #print('variance')
    #h = hf[0]
    idcs = {}
    hfun = hxs[0]
    nfun = h.shape[0]
    for ncol in range(ncols):
        hx = hxs[ncol]
        nx = h.shape[ncol]

        Eygx = np.zeros(nx)
        hxms = [hxm[i] for i in range(ncols) if i != ncol]
        XX = meshgrid(*hxms, indexing = 'ij')
        for ix in range(nx):
            ht = np.take(h, ix, axis=ncol) # np.take(h,ix, axis = ncol)
            Eygx[ix] = sum(XX[0]*ht)/sum(ht)
        
        ht = h.sum(axis = tuple([i for i in range(ncols) if i != ncol]))
        hx = hxm[ncol]
        xme = np.nansum(Eygx*ht)/np.nansum(ht)
        xva = np.nansum((Eygx-xme)**2*ht)/np.nansum(ht)
        #print(xme)
        idcs[alabels[ncol]] = xva
    return idcs

In [195]:
sensitivityidcs(mesT[:,1]/mesT[:,2], pddataT, labelsmod,
                ylabel = 'Vsp/Vsh', ns = [31,11,15,12])

/export/home1/users/bssn/serna/environments/neuron/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


{'Vsp/Vsh': 0.13450529244822546,
 'A1': 0.1519093137007572,
 'Dss': 0.03560843536388895,
 'Rneck': 0.9437078457823006}

In [176]:
labelst = labelsmod


#Sobol indices
def returnSidcs(labels, ns = 30):
    datI = dis[labels[0]]
    for l in labels[1:]:
        datI = column_stack((datI,dis[l]))

    datI = pd.DataFrame(datI, columns =labels )

    nmod = len(labels)+1
    Sidx = {}

    Sidx['Vsoma'] = sensitivityidcs(mesT[:,3], pddataT, labels,
                    ylabel = 'Vsoma', ns = [ns]*nmod)
    Sidx['Vsp'] = sensitivityidcs(mesT[:,1], pddataT, labels,
                    ylabel = 'Vsp', ns = [ns]*nmod)
    Sidx['Vsp-Vsh'] = sensitivityidcs(mesT[:,1]-mesT[:,2], pddataT,labels,
                    ylabel = 'Vsp-Vsh', ns = [ns]*nmod)
    Sidx['Vsp/Vsh'] =sensitivityidcs(mesT[:,1]/mesT[:,2], pddataT,labels,
                    ylabel = 'Vsp/Vsh', ns = [ns]*nmod)
    Sidx['C'] =sensitivityidcs(mesT[:,4], pddataT, labels,
                    ylabel = 'C', ns = [ns]*nmod)
    Sidx['Sv'] =sensitivityidcs(mes1[:,1]/mes01[:,1], datI, labels,
                    ylabel = 'Sv', ns = [ns]*nmod)
    Sidx['Sc'] = sensitivityidcs(mes1[:,4]/mes01[:,4], datI, labels,
                    ylabel = 'Sc', ns = [ns]*nmod)
    Sidx['St'] = sensitivityidcs(times/times0, datI, labels,
                    ylabel = 'St', ns = [ns]*nmod)
    return Sidx

Sidcsmod = returnSidcs(labelsmod)

/export/home1/users/bssn/serna/environments/neuron/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars


In [177]:
Sidcsmorph = returnSidcs(labelmorph, ns = 11)

/export/home1/users/bssn/serna/environments/neuron/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
mes1.shape, pddataT.shape

((37, 9), (384, 30))

In [178]:

labels = labelsmod
print('Quantity\t '+'      '.join([label for label in labels]))
for key in Sidcsmod.keys():
    Sidcs = Sidcsmod[key]
    formattext = '{:.4f}  '*3+' '
    sobol = np.array([Sidcs[key2] for key2 in labels])
    ssum = sobol.sum()
    sobol = sobol/ssum
    print('{}\t\t '.format(key)+formattext.format(*sobol)+'   {:.4f}'.format(ssum) )

Quantity	 A1      Dss      Rneck
Vsoma		 0.7657  0.0943  0.1400      1.1797
Vsp		 0.7613  0.0843  0.1544      0.9721
Vsp-Vsh		 0.5422  0.0872  0.3706      0.9249
Vsp/Vsh		 0.2055  0.0602  0.7343      1.3026
C		 0.7589  0.0665  0.1746      1.1936
Sv		 0.3513  0.3037  0.3450      1.7153
Sc		 0.2569  0.2906  0.4526      1.9095
St		 0.4338  0.3763  0.1899      1.7147


In [179]:
labels = labelsmod
print('Quantity \t'+' \t'.join([label for label in labels]))
for key in Sidcsmod.keys():
    Sidcs = Sidcsmod[key]
    formattext = '{:.4f} \t'*3+' '
    sobol = np.array([Sidcs[key2] for key2 in labels])
    ssum = sobol.sum()
    sobol = sobol/ssum
    print('{} \t'.format(key)+formattext.format(*sobol)+'{:.4f} \t'.format(ssum) )

Quantity 	A1 	Dss 	Rneck
Vsoma 	0.7657 	0.0943 	0.1400 	 1.1797 	
Vsp 	0.7613 	0.0843 	0.1544 	 0.9721 	
Vsp-Vsh 	0.5422 	0.0872 	0.3706 	 0.9249 	
Vsp/Vsh 	0.2055 	0.0602 	0.7343 	 1.3026 	
C 	0.7589 	0.0665 	0.1746 	 1.1936 	
Sv 	0.3513 	0.3037 	0.3450 	 1.7153 	
Sc 	0.2569 	0.2906 	0.4526 	 1.9095 	
St 	0.4338 	0.3763 	0.1899 	 1.7147 	


In [180]:
labels = labelmorph
print('Quantity\t '+'      '.join([label for label in labels]))
for key in Sidcsmorph.keys():
    Sidcs = Sidcsmorph[key]
    formattext = '{:.4f}  '*len(labels)+' '
    sobol = np.array([Sidcs[key2] for key2 in labels])
    ssum = sobol.sum()
    sobol = sobol/ssum
    print('{}\t\t '.format(key)+formattext.format(*sobol)+'   {:.4f}'.format(ssum) )

Quantity	 A1      maxDhead      Dss      Lneck      Lhead      meanDneck_trunc70pc
Vsoma		 0.5168  0.2913  0.0440  0.0166  0.0127  0.1185      1.6191
Vsp		 0.6040  0.2482  0.0370  0.0708  0.0245  0.0156      1.1179
Vsp-Vsh		 0.5524  0.0940  0.0386  0.2091  0.0220  0.0839      0.7702
Vsp/Vsh		 0.1001  0.1404  0.0176  0.3832  0.0081  0.3506      1.4596
C		 0.6496  0.0931  0.0644  0.0709  0.0536  0.0684      1.2505
Sv		 0.1620  0.1012  0.1405  0.2475  0.1723  0.1764      1.8657
Sc		 0.2510  0.2326  0.1962  0.1311  0.0948  0.0943      1.5629
St		 0.2416  0.1918  0.2674  0.0528  0.1692  0.0772      1.7383


In [181]:
labels = labelmorph
print('Quantity '+' '.join([label for label in labels]))
for key in Sidcsmorph.keys():
    Sidcs = Sidcsmorph[key]
    formattext = '{:.4f} '*len(labels)+' '
    sobol = np.array([Sidcs[key2] for key2 in labels])
    ssum = sobol.sum()
    sobol = sobol/ssum
    print('{} '.format(key)+formattext.format(*sobol)+'{:.4f} '.format(ssum) )

Quantity A1 maxDhead Dss Lneck Lhead meanDneck_trunc70pc
Vsoma 0.5168 0.2913 0.0440 0.0166 0.0127 0.1185  1.6191 
Vsp 0.6040 0.2482 0.0370 0.0708 0.0245 0.0156  1.1179 
Vsp-Vsh 0.5524 0.0940 0.0386 0.2091 0.0220 0.0839  0.7702 
Vsp/Vsh 0.1001 0.1404 0.0176 0.3832 0.0081 0.3506  1.4596 
C 0.6496 0.0931 0.0644 0.0709 0.0536 0.0684  1.2505 
Sv 0.1620 0.1012 0.1405 0.2475 0.1723 0.1764  1.8657 
Sc 0.2510 0.2326 0.1962 0.1311 0.0948 0.0943  1.5629 
St 0.2416 0.1918 0.2674 0.0528 0.1692 0.0772  1.7383 
